In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from tensorflow.keras.utils import to_categorical
import tensorflow as tf

Resize

In [ ]:
path = "./TrainingDataSet/"
dir_list = os.listdir(path)
for i in dir_list:
    if(i[0] != '.'):
        img_files = os.listdir(path+i)
        for j in img_files:
            if(j[0] != '.'):
                image = Image.open(path+i+"/"+j).convert('L')
                new_image = image.resize((100, 100))
                new_image.save(path+i+"/"+j)

In [ ]:
label = sorted(["fake","real"])

create array

In [ ]:
path = "./TrainingDataSet/"
images_np = tf.keras.utils.image_dataset_from_directory(path,color_mode="grayscale",image_size=(100,100))

Display first batch

In [ ]:
data_iterator = images_np.as_numpy_iterator()
batch = data_iterator.next()

idx = np.random.randint(len(batch[0]) - 9)
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(batch[0][i])
    plt.xlabel(label[batch[1][i]])
plt.show()

Normalize

In [ ]:
images_np = images_np.map(lambda x, y: (x/255,y))

Split Data

In [ ]:
train_size = int(len(images_np)*0.7)
val_size =  int(len(images_np)*0.2)
test_size = int(len(images_np)*0.1)

train = images_np.take(train_size)
val = images_np.skip(val_size).take(val_size)
test = images_np.skip(test_size).take(test_size)

Build Deep Learning Model

In [ ]:
from tensorflow.keras import Sequential, layers

def build_model():
    model = Sequential()
    # 32 output units, 3 filter size
    model.add(layers.Conv2D(16, 3, 1, activation='relu', padding='same', input_shape=(100, 100, 1)))
    model.add(layers.MaxPooling2D())
    # 32 output units, 5 filter size
    model.add(layers.Conv2D(32, 3, 1, activation='relu'))
    model.add(layers.MaxPooling2D())

    model.add(layers.Conv2D(16, 3, 1, activation='relu'))
    model.add(layers.MaxPooling2D())

    model.add(layers.Flatten())

    model.add(layers.Dense(100, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(
        optimizer ='adam',
        loss = tf.losses.BinaryCrossentropy(),
        metrics = ['accuracy'])
    return model

# create model
model = build_model()

# see model details
model.summary()

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard("logs")

Train

In [ ]:
%%time
history = model.fit(train, epochs=20,validation_data=val, callbacks= [tensorboard_callback])
model.save('./models/Deepfake_Detection.keras')

print("\ninitial accuracy: {} | latest accuracy: {}".format(history.history["accuracy"][0], history.history["accuracy"][-1]))
print("initial loss: {} | latest loss: {}".format(history.history["loss"][0], history.history["loss"][-1]))

Plot Performance

In [ ]:
fig, axis = plt.subplots(1, 2, figsize=(10,4))
axis[0].plot(history.history["accuracy"])
axis[0].set_title("accuracy over epochs")
axis[1].plot(history.history["loss"])
axis[1].set_title("loss over epochs")
plt.show()

Evaluate

In [ ]:
test_path = '/Users/rediettadesse/Desktop/AI_projects/Deepfake Detection/TrainingDataSet/fake/fake-826.png'
image = tf.keras.utils.load_img(test_path,color_mode="grayscale",target_size=(100,100))
input_arr = tf.keras.utils.img_to_array(image)
input_arr = np.array([input_arr])/255.0
predictions = model.predict(input_arr)
plt.figure(figsize=(10,10))
plt.imshow(image)
plt.title("Predicted : "+label[round(predictions[0][0])])
plt.show()